In [45]:
import pandas as pd
import requests
import xml.etree.ElementTree as ET
import xml.dom.minidom
import json
import csv
import datetime as dt


In [46]:
myToken = 'NLOD'
myUrl = 'https://api.entur.io/stop-places/v1/read/stop-places?transportModes=BUS'
head1 = {'Authorization': 'token {}'.format(myToken)}
response = requests.get(myUrl, headers=head1)
# Print raw text content of the response to check the format, this is how I found out that it´s xml
#print(response.headers['Content-Type'])

In [47]:
#print(response.content)

In [48]:

#print(response.text) 

In [52]:
data = response.json() #converts JSON to Python dictionary
print(data)



[{'id': 'NSR:StopPlace:1000', 'validBetween': [{'modification': 'NEW', 'status_BasicModificationDetailsGroup': 'ACTIVE', 'publication': 'PUBLIC', 'fromDate': '2017-06-19T21:14:34.1+02:00'}], 'created': '2017-06-19T21:14:34.1+02:00', 'changed': '2024-01-01T02:24:58.536+01:00', 'modification': 'NEW', 'version': '1', 'status_BasicModificationDetailsGroup': 'ACTIVE', 'publication': 'PUBLIC', 'keyList': {'keyValue': [{'key': 'imported-id', 'value': 'OST:StopPlace:01223282,OST:StopPlace:1223282'}, {'key': 'IS_PARENT_STOP_PLACE', 'value': 'false'}]}, 'name': {'value': 'Sentvedt', 'lang': 'nor'}, 'centroid': {'modification': 'NEW', 'status_BasicModificationDetailsGroup': 'ACTIVE', 'publication': 'PUBLIC', 'location': {'longitude': 11.379885, 'latitude': 59.66847}}, 'accessibilityAssessment': {'id': 'NSR:AccessibilityAssessment:1000', 'modification': 'NEW', 'version': '1', 'status_BasicModificationDetailsGroup': 'ACTIVE', 'publication': 'PUBLIC', 'mobilityImpairedAccess': 'UNKNOWN', 'limitation

In [53]:
#print(data[0].keys())  #show the keys so that you can filter later

In [54]:
keys_to_keep = ['id', 'created', 'name', 'centroid', 'weighting']

processed_data = []
for entry in data:
    # Extract the 'value' from 'name'
    entry['name'] = entry.get('name', {}).get('value', None) 
    
    # Extract longitude and latitude from centroid['location']
    if 'centroid' in entry and 'location' in entry['centroid']:
        location = entry['centroid']['location']
        entry['longitude'] = location.get('longitude', None)
        entry['latitude'] = location.get('latitude', None)
        
        
        del entry['centroid']  # Remove the entire centroid field
    
    # Keep only the specified keys
    filtered_entry = {key: entry.get(key, None) for key in keys_to_keep}
    processed_data.append(filtered_entry)

# Convert the processed data into a DataFrame
df = pd.DataFrame(processed_data)

df.head()

,id,created,name,centroid,weighting
0,NSR:StopPlace:1000,2017-06-19T21:14:34.1+02:00,Sentvedt,None,INTERCHANGE_ALLOWED
1,NSR:StopPlace:10000,2017-06-19T21:24:52.897+02:00,Grøtholm,None,INTERCHANGE_ALLOWED
2,NSR:StopPlace:10001,2017-06-19T21:24:52.851+02:00,Venstad vegkryss,None,INTERCHANGE_ALLOWED
3,NSR:StopPlace:10002,2017-06-19T21:24:52.942+02:00,Husum,None,INTERCHANGE_ALLOWED
4,NSR:StopPlace:10003,2017-06-19T21:24:52.985+02:00,Langåsen,None,INTERCHANGE_ALLOWED
